## Trabajo práctico final
Victoria Aguirre, Mauricio Enrich, Milagros Maurer y Felipe Pasquet.

### Importación de librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import json
import pandas as pd
import seaborn as sns
import random
from collections import defaultdict, Counter
from datetime import datetime, timedelta
import time
ubicacion_items = defaultdict(list)
totales_racks = Counter()

### Carga de datos

In [ ]:
with open('stock.json','r',encoding= 'utf-8') as archivo:
  stock = json.load(archivo)

In [ ]:
with open('backlog.json','r',encoding = 'utf-8') as archivo:
  backlog = json.load(archivo)

Veamos que estructura tienen los datos.

In [ ]:
stock['Rack_00001']

{'Cara_1': [{'Inventory ID': '1U0KF6M5H964WNX',
   'Cantidad': 4,
   'Nivel': 2,
   'Posicion': 2},
  {'Inventory ID': '1U0KF6M5H964WNX',
   'Cantidad': 1,
   'Nivel': 4,
   'Posicion': 4},
  {'Inventory ID': '23K4GP7JVCG918G',
   'Cantidad': 2,
   'Nivel': 3,
   'Posicion': 1},
  {'Inventory ID': '530INC74HG3R08R',
   'Cantidad': 1,
   'Nivel': 3,
   'Posicion': 3},
  {'Inventory ID': '6OM5CVCNUT5U3VW',
   'Cantidad': 2,
   'Nivel': 2,
   'Posicion': 3},
  {'Inventory ID': '7438YZGRK3Y43BH',
   'Cantidad': 1,
   'Nivel': 5,
   'Posicion': 2},
  {'Inventory ID': '9ER87IHTWAOBWZA',
   'Cantidad': 2,
   'Nivel': 2,
   'Posicion': 1},
  {'Inventory ID': '9ER87IHTWAOBWZA',
   'Cantidad': 3,
   'Nivel': 2,
   'Posicion': 4},
  {'Inventory ID': '9ER87IHTWAOBWZA',
   'Cantidad': 2,
   'Nivel': 4,
   'Posicion': 2},
  {'Inventory ID': '9UM8P51BTAL04EU',
   'Cantidad': 1,
   'Nivel': 2,
   'Posicion': 2},
  {'Inventory ID': 'BCW1MVAP2EYM650',
   'Cantidad': 1,
   'Nivel': 2,
   'Posicion': 4},


In [ ]:
backlog['orders'][0]

{'order_id': 'ORD_013387_LXJY4YBS_000',
 'item_id': 'LXJY4YBSWCX3KBF',
 'quantity': 1,
 'creation_date': '2025-10-09T00:00:04.885077',
 'due_date': '2025-10-09T21:00:00.885077'}

In [ ]:
# Seteamos semilla
np.random.seed(100)
random.seed(100)

## Heurística

### Limpieza de backlog

Durante el problema, nos dimos cuenta que muchas de las órdenes no cuentan con el stock suficiente para ser suplidas. Es por eso, que dado el backlog original, decidimos hacer una copia nueva, donde unicamente tenga las órdenes que sí tienen stock.

In [ ]:
# pedidos_items devuleve, por item, cuántos pedidos hay de ese item
# devuelve por item, las ordenes de ese item

pedidos_items = {}
pedidos_por_item = {}


for i in range(len(backlog['orders'])):

  item = backlog['orders'][i]['item_id']

  if(item not in pedidos_items):

    pedidos_items[item] = 1
    pedidos_por_item[item] = [backlog['orders'][i]]

  else:

    pedidos_items[item] += 1
    pedidos_por_item[item].append(backlog['orders'][i])

In [ ]:
# se calcula cuanto stock hay de cada item, recorriendo todos los racks y todas sus caras

stock_items = {}

for rack in stock:

    for cara in stock[rack]:

        for i in range(len(stock[rack][cara])):

            item_id = stock[rack][cara][i]['Inventory ID']
            cantidad = stock[rack][cara][i]['Cantidad']

            if item_id not in stock_items and cantidad > 0:

                stock_items[item_id] = cantidad

            elif cantidad > 0:

                stock_items[item_id] += cantidad

In [ ]:
backlog_preprocesado = []

for item in pedidos_items:
    # si el item no está en el stock, el stock es 0
    stock_item_i = stock_items.get(item, 0)
    pedido_item_i = pedidos_items[item]

    if stock_item_i - pedido_item_i < 0: # no hay suficiente stock del item

        cant = pedido_item_i - stock_item_i # cantidad de pedidos que si se pueden suplir

        # si hay menos stock que pedidos, tomamos los que  se pueden cumplir (elegimos quedarnos con los primeros)
        backlog_preprocesado.extend(pedidos_por_item[item][:stock_item_i])

    else:

      backlog_preprocesado.extend(pedidos_por_item[item][:stock_item_i])

print("La cantidad de órdenes a entregar es de:" ,len(backlog_preprocesado))

La cantidad de órdenes a entregar es de: 171746


Vamos a trabajar con el backlog ordenado por due_date, ya que queremos darle prioridad a aquellas que están próximas a vencerse.

In [ ]:
backlog_ordenado = sorted(backlog_preprocesado, key=lambda x: datetime.fromisoformat(x["due_date"]))

### SS (Shelf Selection)

In [ ]:
def construir_ubicacion(stock):

    global ubicacion_items, totales_racks #variables globales del problema
    ubicacion_items = defaultdict(list) # creamos un diccionario para que, para cada item nos devuelva todas las ubicaciones (rack y cara) en las que se encuentra
    totales_racks = defaultdict(int)

    for rack, caras in stock.items():
        for cara, items in caras.items():
            for entry in items:

                item = entry["Inventory ID"]
                cantidad = int(entry["Cantidad"])

                if cantidad <= 0:
                    continue

                ubicacion_items[item].append({
                    "rack": rack,
                    "cara": cara,
                    "cantidad": cantidad
                })

                totales_racks[rack] += cantidad

In [ ]:
construir_ubicacion(stock)

Acá se puede ver, como dado un ítem, nos devuelve que se encuentra en el rack 01575 en la cara 1, y en el rack 01972 eb la cara 1. En ambos, en 1 cantidad.

In [ ]:
ubicacion_items['000A2DTFPGC7BYO']

[{'rack': 'Rack_01575', 'cara': 'Cara_1', 'cantidad': 1},
 {'rack': 'Rack_01972', 'cara': 'Cara_1', 'cantidad': 1}]

Con la función **`definir_racks(orden, racks_calientes, racks_tibios)`** nuestro objetivo es, dado una orden definir un rack y una cara para pickear esa orden, tratando de dismunir el uso de racks tibios y fríos, priorizando el uso de racks calientes. Racks calientes es una matriz de **cant_racks_totales x 4**, donde la posición i,j es 1 si el rack fue usado sobre la ola, 0 en caso contrario. Y los racks tibios son los racks usados en la ola anterior.

In [ ]:
def definir_rack(orden, racks_calientes, racks_tibios):

    global ubicacion_items, totales_racks

    item = orden["item_id"] #item a pickear
    order_id = orden["order_id"]

    rack = ""

    if ubicacion_items[item] == []: # Me fijo si hay stock del item, esto es por si no hay stock
      return None, 0

    stock_item = 0

    for i in range(len(ubicacion_items[item])):
      stock_item += ubicacion_items[item][i]['cantidad']

    if stock_item == 0: #Cuando te quedaste sin stock porque vas completando las ordenes
      return None, 0

    #Si llegamos aca, es porque hay stock

    #CHEQUEO SI HAY RACKS CALIENTES PARA SATISFACER LA ORDEN

    mejor_clasificacion = 50
    index = -1
    for i in range(len(ubicacion_items[item])):

      r = int(ubicacion_items[item][i]['rack'].split('_')[1]) #rack donde se ecnuentra
      c = int(ubicacion_items[item][i]['cara'].split('_')[1]) #cara donde se encuentra

      if ubicacion_items[item][i]['cantidad'] == 0:
        continue

      if racks_calientes[r-1][c-1] == 1:
        clasificacion_actual = 0 # 0 es el costo de ni siquiera moverme del rack en el que estaba

      elif sum(racks_calientes[r-1]) >= 1:
        clasificacion_actual = 1 # 1 es el costo de quedarme en el mismo rack pero cambiar de cara

      elif racks_tibios[r-1] == 1:
        clasificacion_actual = 3 # 3 es el costo de utilizar un rack tibio
      else:
        clasificacion_actual = 5 # 5 si utilizo un rack frío

      if clasificacion_actual < mejor_clasificacion: #Agarro la que me genere un menor costo
        mejor_clasificacion = clasificacion_actual
        index = i

    if index == -1:
      return None, 0

    rack = ubicacion_items[item][index]['rack']
    cara = ubicacion_items[item][index]['cara']
    r = int(rack.split('_')[1])
    c = int(cara.split('_')[1])

    if mejor_clasificacion == 0:
      ubicacion_items[item][index]['cantidad'] -= 1
      costo = 0
      tipo_rack = "Rack Caliente"
    elif mejor_clasificacion == 1:
      ubicacion_items[item][index]['cantidad'] -= 1
      costo = 1
      tipo_rack = "Rack Caliente"
    elif mejor_clasificacion == 3:
      ubicacion_items[item][index]['cantidad'] -= 1
      costo = 3
      tipo_rack = "Rack Tibio"
    elif mejor_clasificacion == 5:
      ubicacion_items[item][index]['cantidad'] -= 1
      costo = 5
      tipo_rack = "Rack Frio"


    totales_racks[rack] -= 1 #Le resto el ítem que acabamos de agarrar correspondiente a esta orden
    racks_calientes[r - 1][c - 1] = 1 #el rack utilizado se vuelve caliente

    tarea = (orden, item, rack, cara, tipo_rack)

    return tarea, costo

In [ ]:
#Inicializamos variables globales

cant_ordenes_vencidas = 0

In [ ]:
def procesar_backlog(racks_calientes, racks_tibios,faltantes, hora_actual): #este tomaba a backlog
  global cant_ordenes_vencidas

  tareas = []
  costo_total = 0
  penalizacion_total = 0

  if faltantes <= 0:
      return tareas, costo_total

  # Ordenar backlog por fecha de vencimiento (más urgentes primero) (CREO QUE NO HACE FALTA)
  ordenes_ordenadas = sorted(
      backlog_final['orders'],
      key=lambda x: datetime.fromisoformat(x["due_date"])
  )

  # Tomar solo las más próximas a vencer
  ordenes_urgentes = ordenes_ordenadas[:faltantes]
  ordenes_asignadas_ids = set()

  for orden in ordenes_urgentes:

      # Clasificar y asignar igual que las pendientes

      tarea_urgente, costo = definir_rack(orden,racks_calientes, racks_tibios)

      if tarea_urgente:
          tareas.append(tarea_urgente)
          costo_total += costo
          ordenes_asignadas_ids.add(orden['order_id'])

  backlog_final['orders'] = [o for o in backlog_final['orders'] if o['order_id'] not in ordenes_asignadas_ids] #elimino del backlog las ordenes ya asignadas

  while len(tareas) < faltantes and backlog_final["orders"]:
      agregado = False  # marca si se agregó algo en esta pasada
      for orden in list(backlog_final["orders"]):  # iterar sobre copia para poder eliminar
          tarea_nueva, costo = definir_rack(orden, racks_calientes, racks_tibios)
          if tarea_nueva:
              tareas.append(tarea_nueva)
              costo_total += costo
              backlog_final["orders"].remove(orden)
              agregado = True
              ordenes_asignadas_ids.add(orden['order_id'])

              due_date = datetime.fromisoformat(orden["due_date"])
              if hora_actual > due_date: # Si la orden se venció, se penaliza
                  penalizacion_total += 100
                  cant_ordenes_vencidas +=1

              if len(tareas) >= faltantes:
                  break
      if not agregado:
          # si no se pudo agregar ninguna orden más (no hay stock válido)
          break

  costo_total += penalizacion_total
  backlog_final['orders'] = [o for o in backlog_final['orders'] if o['order_id'] not in ordenes_asignadas_ids] #elimino del backlog las ordenes ya asignadas

  return tareas, costo_total

In [ ]:
def procesar_pendientes(pendientes, racks_calientes, racks_tibios):
  tareas = []
  costo = 0

  for orden_pendiente in pendientes:

    tarea, costo_tarea = definir_rack(orden_pendiente, racks_calientes, racks_tibios)

    if tarea:
      tareas.append(tarea)
      costo += costo_tarea

  return tareas, costo

In [ ]:
def actualizar_tibios(tareas, racks_tibios, cant_max_tibios):

  for tarea in tareas:

    nro_rack = int(tarea[2].split('_')[1])
    racks_tibios[nro_rack-1] = 1

  if sum(racks_tibios) > cant_max_tibios:

    cant = sum(racks_tibios) - cant_max_tibios

    for i in range(len(racks_tibios)):

      if racks_tibios[i] == 1:

        racks_tibios[i] = 0
        cant -= 1
        if cant == 0:
          break

In [ ]:
def SS(stock,pendientes, N, racks_tibios,cant_racks, hora_actual): #este tomaba a backlog

  racks_calientes = np.zeros((cant_racks, 4))
  # [[1,0,0,0],[0,0,0,0]], tamaño de la matriz 2087x4
  # N - pendientes + no_cumplidas

  cant_max_tibios = int(0.2 * cant_racks)

  tareas_pend, costo_pend = procesar_pendientes(pendientes, racks_calientes, racks_tibios) # primero asignamos las ordenes pendientes

  faltantes = max(0, N - len(tareas_pend))
  tareas_urgentes, costo_urgente = procesar_backlog(racks_calientes, racks_tibios, faltantes, hora_actual) # asignamos N-len(pendientes)

  tareas = tareas_pend + tareas_urgentes
  costo = costo_pend + costo_urgente

  actualizar_tibios(tareas, racks_tibios, cant_max_tibios) # actualizo racks tibios

  resultado = defaultdict(list)
  for tarea in tareas:
    resultado[(tarea[2], tarea[3])].append(tarea[0]['order_id'])

  return resultado, costo # devolvemos pool de tareas y costo total del ciclo

In [ ]:
backlog_preprocesado

[{'order_id': 'ORD_013387_LXJY4YBS_000',
  'item_id': 'LXJY4YBSWCX3KBF',
  'quantity': 1,
  'creation_date': '2025-10-09T00:00:04.885077',
  'due_date': '2025-10-09T21:00:00.885077'},
 {'order_id': 'ORD_013387_LXJY4YBS_001',
  'item_id': 'LXJY4YBSWCX3KBF',
  'quantity': 1,
  'creation_date': '2025-10-09T00:00:04.885077',
  'due_date': '2025-10-09T06:00:00.885077'},
 {'order_id': 'ORD_004503_0N9X97RY_000',
  'item_id': '0N9X97RYEKKHWE1',
  'quantity': 1,
  'creation_date': '2025-10-09T00:00:09.143135',
  'due_date': '2025-10-09T21:00:00.143135'},
 {'order_id': 'ORD_004503_1H0SDN0A_000',
  'item_id': '1H0SDN0ATVN17AR',
  'quantity': 1,
  'creation_date': '2025-10-09T00:00:09.143135',
  'due_date': '2025-10-09T21:00:00.143135'},
 {'order_id': 'ORD_004503_T28BROHN_000',
  'item_id': 'T28BROHNZ5LWGKC',
  'quantity': 1,
  'creation_date': '2025-10-09T00:00:09.143135',
  'due_date': '2025-10-09T16:00:00.143135'},
 {'order_id': 'ORD_004503_T28BROHN_001',
  'item_id': 'T28BROHNZ5LWGKC',
  'quan

In [ ]:
backlog_total = backlog_preprocesado
resultados = []
CANT_CICLOS = 288
cant_racks = int(list(stock.keys())[-1].split('_')[1])
racks_tibios = [0] * cant_racks
fecha_inicial = datetime(2025, 10, 9, 0, 0, 0)
suma_ordenes_cumplidas = 0
costo_final = 0
ordenes_restantes = len(backlog_preprocesado)
costos_promedios = []
inicio_total = time.time()

backlog_final = {"orders": []} # en este backlog se van guardando las ordenes ya creadas y listas para ser cumplidas

np.random.seed(42)

caras_totales = []
racks_totales = set()
caras_sobre_racks = []

for ciclo in range(1, CANT_CICLOS + 1):
    hora_actual = fecha_inicial + timedelta(minutes=5 * ciclo)
    print(f"\n Ciclo {ciclo} - Hora simulada: {hora_actual.strftime('%H:%M')}")

    ordenes_disponibles = [
    o for o in backlog_total
    if (datetime.fromisoformat(o["creation_date"]) <= hora_actual and datetime.fromisoformat(o["creation_date"])> hora_actual - timedelta(minutes=5))
    ] # me guardo las ordenes que fueron creadas en los ultimos 5 minutos

    ids_actuales = {o["order_id"] for o in backlog_final["orders"]}
    nuevas_ordenes = [o for o in ordenes_disponibles if o["order_id"] not in ids_actuales]

    if nuevas_ordenes:
        backlog_final["orders"].extend(nuevas_ordenes)

    numero = random.randint(1000, 2000)
    N = min(len(backlog_final["orders"]), numero)

    cant_pendientes = random.randint(0, 100)

    pendientes = random.sample(backlog_final["orders"], min(cant_pendientes, len(backlog_final["orders"])))
    pendientes_ids = {p["order_id"] for p in pendientes}
    backlog_final["orders"] = [orden for orden in backlog_final["orders"] if orden["order_id"] not in pendientes_ids]


    print(f"Preparando ejecución... N={N}, Pendientes={len(pendientes)}")


    try:
        resultado, costo_total = SS(stock,pendientes, N, racks_tibios, cant_racks, hora_actual)

        if len(resultado) > 0:
          caras_usadas = len(resultado)  # (rack, cara)
          ppf = N/caras_usadas

          racks_usados = set()
          for res in resultado:
            racks_usados.add(res[0])

          caras_totales.append(len(resultado))
          cant_racks_usados = len(list(racks_usados))
          caras_sobre_racks.append(len(resultado)/cant_racks_usados)

        else:
          ppf = 0

        racks_totales = racks_totales.union(racks_usados)


        print(f"Ciclo {ciclo} OK → {N} tareas | Costo total: {costo_total:.0f} | PPF: {ppf:.2f}")

        resultados.append({
            "ciclo": ciclo,
            "hora": hora_actual.strftime("%H:%M"),
            "tareas": N,
            "costo": costo_total,
            "ppf": ppf,
            "pendientes": len(pendientes),
            "ordenes_restantes": ordenes_restantes,
            "costo_por_paquete" : costo_total / N
        })

        suma_ordenes_cumplidas += N
        costo_final += costo_total
        ordenes_restantes -= N
        print(f"Ordenes restantes: {ordenes_restantes}")
    except Exception as e:
        print(f" Error en ciclo {ciclo}: {type(e).__name__} → {e}")
        break



# === Calcular promedio diario de PPF ===
ppf_valores = [r["ppf"] for r in resultados if r["ppf"] > 0]
if len(ppf_valores) > 0:
    promedio_ppf = sum(ppf_valores) / len(ppf_valores)
else:
    promedio_ppf = 0

# === Calcular promedios de costo por paquete para cada ciclo ===
promedios_valores = [r['costo_por_paquete'] for r in resultados]

fin_total = time.time()
tiempo_total = fin_total - inicio_total
tiempo_promedio_por_ciclo = tiempo_total / CANT_CICLOS
costo_prom_por_orden = costo_final / suma_ordenes_cumplidas
tasa_racks_utilizados = (len(list(racks_totales)) / cant_racks)*100
#tasa_caras_utilizados = (total_caras_usadas / cant_caras)
tasa_cumplimiento_SLA = ((suma_ordenes_cumplidas-cant_ordenes_vencidas) / suma_ordenes_cumplidas) * 100

print(f"\n Ordenes totales cumplidas: {suma_ordenes_cumplidas}")
print(f"\n Costo final: {costo_final}")
print(f"\n Promedio de costo por paquete por ciclo: {promedios_valores}")
print(f"\n Racks únicos utilizados: {cant_racks_usados}({tasa_racks_utilizados:.2f}%)")
print(f"\n Promedio de cantidad de caras usadas en un ciclo: {sum(caras_totales)/len(caras_totales):.2f}")
print(f"\n Promedio caras usadas por racks usados: {np.mean(caras_sobre_racks):.2f}")
print(f"\n Promedio diario de Picks per Face: {promedio_ppf:.2f}")
print(f"\n Costo promedio por orden: {costo_prom_por_orden:.2f}")
print(f"\n Tasa de cumplimiento SLA: {tasa_cumplimiento_SLA}%")
print(f"\n Tiempo promedio de ejecución por ciclo: {tiempo_promedio_por_ciclo:.2f} segundos")


 Ciclo 1 - Hora simulada: 00:05
Preparando ejecución... N=962, Pendientes=58
Ciclo 1 OK → 962 tareas | Costo total: 2467 | PPF: 1.75
Ordenes restantes: 170784

 Ciclo 2 - Hora simulada: 00:10
Preparando ejecución... N=1088, Pendientes=20
Ciclo 2 OK → 1088 tareas | Costo total: 2414 | PPF: 1.81
Ordenes restantes: 169696

 Ciclo 3 - Hora simulada: 00:15
Preparando ejecución... N=762, Pendientes=55
Ciclo 3 OK → 762 tareas | Costo total: 1784 | PPF: 1.77
Ordenes restantes: 168934

 Ciclo 4 - Hora simulada: 00:20
Preparando ejecución... N=759, Pendientes=56
Ciclo 4 OK → 759 tareas | Costo total: 1740 | PPF: 1.82
Ordenes restantes: 168175

 Ciclo 5 - Hora simulada: 00:25
Preparando ejecución... N=928, Pendientes=38
Ciclo 5 OK → 928 tareas | Costo total: 1918 | PPF: 1.99
Ordenes restantes: 167247

 Ciclo 6 - Hora simulada: 00:30
Preparando ejecución... N=781, Pendientes=48
Ciclo 6 OK → 781 tareas | Costo total: 1866 | PPF: 1.71
Ordenes restantes: 166466

 Ciclo 7 - Hora simulada: 00:35
Prepa